### Image Deblurring and QR Factorizations
#### DDA3005 Course Project
Chen Lexuan             122090029

In [ ]:
import os
import time
from PIL import Image

import numpy as np
from numpy import asarray
from matplotlib import image
import matplotlib.pyplot as plt

import scipy.linalg
from scipy import linalg
from scipy.linalg import solve
from scipy.sparse import linalg as splinalg
import matplotlib.pyplot as plt

In [ ]:
"""
part(a)

    Load image 
    Construct blurring kernels
    Derive blurring images

    When processing the problem, we find the shape of images in tested_images are not unified. For example, '1250_m3_original.png' has channel parameter, but '1024_books_original.png' not. To make the structure of the code clearer, we uniformly converted the images to single-channel grayscale images when loading and saving them.
"""

In [ ]:
def load_image(path):
    """Load image and convert to grayscale, normalize to [0,1] range"""
    # Load image and convert to grayscale
    img = Image.open(path).convert('L')

    # Convert to numpy array and normalize to [0,1]
    img_array = np.array(img).astype(np.float64) / 255.0

    return img_array

def save_image(img, path):
    """Save normalized grayscale image"""
    # Ensure the image is in [0,1] range
    img_normalized = np.clip(img, 0, 1)
    # Convert to uint8 [0,255] range
    img_uint8 = (img_normalized * 255).astype(np.uint8)
    # Save using PIL
    Image.fromarray(img_uint8, mode='L').save(path)

In [ ]:
def build_blur_kernel(n, j, k, upper_triangular=True, extra_subdiagonal=False):
    A = np.zeros((n, n))
    weights = np.array([k - i for i in range(k)])
    weights = (2 / (k * (k + 1))) * weights  # Normalize weights

    for i in range(n):
        for offset in range(len(weights)):
            if upper_triangular:
                if i + offset < n:
                    A[i, i + offset] = weights[offset]
            if extra_subdiagonal:
                if i + offset - j < n and i + offset - j >= 0:
                    A[i, i + offset - j] = weights[offset]
    return A

def apply_blur(X, A_l, A_r):
    """Apply blur using equation (1): A_l @ X @ A_r"""
    return A_l @ X @ A_r

def compute_psnr(original, reconstructed):
    """Compute Peak Signal-to-Noise Ratio"""
    mse = np.mean((original - reconstructed) ** 2)
    if mse == 0:
        return float('inf')
    return 10 * np.log10(1.0 / mse)  # Images are in [0,1] range


def create_blurred_image(img_path, n, j_l=0, k_l=12, j_r=1, k_r=24, pad=True):
    """Create blurred version of an image with kernels of original size
    
    Returns:
        X: Original image
        B: Blurred image
        A_l_orig: Left kernel (original size)
        A_r_orig: Right kernel (original size)
        A_l_padded: Left kernel (padded size) if pad=True, else None
        A_r_padded: Right kernel (padded size) if pad=True, else None
    """
    X = load_image(img_path)

    A_l_orig = build_blur_kernel(n, j_l, k_l,upper_triangular=True, extra_subdiagonal=False)
    A_r_orig = build_blur_kernel(n, j_r, k_r,upper_triangular=False,extra_subdiagonal=True)

    if pad:
        pad_width = max(k_l, k_r)
        X_padded = np.pad(X, pad_width, mode='edge')
        padded_size = X_padded.shape[0]

        A_l_padded = build_blur_kernel(padded_size, j_l, k_l,upper_triangular=True, extra_subdiagonal=False)
        A_r_padded = build_blur_kernel(padded_size, j_r, k_r,upper_triangular=False,extra_subdiagonal=True)

        B_padded = apply_blur(X_padded, A_l_padded, A_r_padded)
        B = B_padded[pad_width:-pad_width, pad_width:-pad_width]

        return X, B, B_padded, A_l_padded, A_r_padded, pad_width
    else:
        B = apply_blur(X, A_l_orig, A_r_orig)
        return X, B, None, A_l_orig, A_r_orig, None

In [ ]:
def process_all_kernels(path, k_r_values, pad=True):
    """Process a single image showing all kernels and blurred results together(all results with kernels"""
    n = int(path.split('/')[1].split('_')[0])
    print(f"Processing image of size {n}x{n}")

    Original_X = load_image(path)

    fig = plt.figure(figsize=(15, 8))

    X, _,_, A_l, A_r,pad_width = create_blurred_image(
        path, n=n, j_l=0, k_l=12, j_r=1, k_r=k_r_values[0], pad=pad
    )

    ax1 = plt.subplot(231)
    im1 = ax1.imshow(A_l, cmap='viridis')
    ax1.set_title('Left Kernel')
    plt.colorbar(im1, ax=ax1)

    for idx, k_r in enumerate(k_r_values, start=2):
        _, _,_, _, A_r, _ = create_blurred_image(
            path, n=n, j_l=0, k_l=12, j_r=1, k_r=k_r, pad=False
        )
        ax = plt.subplot(2, 3, idx)
        im = ax.imshow(A_r, cmap='viridis')
        ax.set_title(f'Right Kernel (k_r={k_r})')
        plt.colorbar(im, ax=ax)

    ax4 = plt.subplot(234)
    ax4.imshow(X, cmap='gray')
    ax4.set_title(f'Original({n}x{n})')
    ax4.axis('off')

    for idx, k_r in enumerate(k_r_values, start=5):
        _, B,_, _, _, _= create_blurred_image(
            path, n=n, j_l=0, k_l=12, j_r=1, k_r=k_r, pad=pad
        )
        ax = plt.subplot(2, 3, idx)
        ax.imshow(B, cmap='gray')
        ax.set_title(f'Blurred (k_r={k_r})')
        ax.axis('off')

    plt.tight_layout()
    plt.show()

    base_name = path.split('/')[1].split('_')[0]


    X,B,_, A_l, A_r,_ = create_blurred_image(
        path, n=n, j_l=0, k_l=12, j_r=1, k_r=k_r_values[0], pad=pad
    )
    save_image(X, f'output2/{base_name}_original.png')

    for k_r in k_r_values:
        _, B, B_padded, A_l, A_r, pad_width = create_blurred_image(
            path, n=n, j_l=0, k_l=12, j_r=1, k_r=k_r, pad=pad
        )
        save_image(B, f'output2/{base_name}_blurred_kr{k_r}.png')
        np.save(f'output2/kernel_left_{base_name}_kr{k_r}.npy', A_l)
        np.save(f'output2/kernel_right_{base_name}_kr{k_r}.npy', A_r)
        
        if B_padded is not None:
            save_image( B_padded, f'output2/{base_name}_blurred_with_padding_kr{k_r}.png')
            pad_info = {'pad_width': pad_width}
            np.save(f'output2/{base_name}_pad_info_kr{k_r}.npy', pad_info)

    print(f"Saved all results for {base_name}")

In [ ]:
def test_blurring(image_paths, k_r_values, pad=False):
    """Process all test images with different blur parameters"""
    os.makedirs('output2', exist_ok=True)

    for path in image_paths:
        print(f"\nProcessing {path}")
        process_all_kernels(path, k_r_values, pad)

def test_blurring_pad(image_paths, k_r_values, pad=True):
    """Process all test images with different blur parameters"""
    os.makedirs('output4', exist_ok=True)

    for path in image_paths:
        print(f"\nProcessing {path}")
        process_all_kernels(path, k_r_values, pad)


    # Test parameters and run
image_paths = [
    'test_images/256_hand_original.png',
    'test_images/640_lion_original.png',
    'test_images/1024_books_original.png',
    'test_images/1250_m1_original.png',
    'test_images/3500_garlic_original.png'
]
k_r_values = [48, 96]

test_blurring(image_paths, k_r_values, pad=False)
print("=====================Results with padding========================")
test_blurring(image_paths, k_r_values, pad=True)

In [ ]:
"""
part(b)

    Solve (1) based on LU factorizations of Aℓ and Ar
    Solve (1) based on QR factorizations of Aℓ and Ar
    Test methods: Observations and explanations
    
"""

In [ ]:
"""
2.1 Construct LU factorizations and QR factorizations
"""
def deblur_lu(B, A_l, A_r,pad_width=None):

    start_time = time.time()

    try:
        P_l, L_l, U_l = scipy.linalg.lu(A_l)
        P_r, L_r, U_r = scipy.linalg.lu(A_r)

        Y1 = solve(L_l, P_l @ B, check_finite=True)

        Z = solve(U_l, Y1, check_finite=True)

        Y2 = solve(L_r.T, P_r.T @ Z.T, check_finite=True)

        X = solve(U_r.T, Y2, check_finite=True).T

        # Handle any potential numerical instabilities
        X = np.clip(X, 0, 1)  # Ensure values are in valid range

        if pad_width is not None:
            X = X[pad_width:-pad_width, pad_width:-pad_width]

    except np.linalg.LinAlgError as e:
        print(f"Linear algebra error in LU method: {str(e)}")
        X = np.zeros_like(B if pad_width is None else B[pad_width:-pad_width, pad_width:-pad_width])
        return X, time.time() - start_time
    except Exception as e:
        print(f"Unexpected error in LU method: {str(e)}")
        X = np.zeros_like(B if pad_width is None else B[pad_width:-pad_width, pad_width:-pad_width])
        return X, time.time() - start_time

    return X, time.time() - start_time


def deblur_qr(B, A_l, A_r,pad_width=None):

    start_time = time.time()

    try:
        Q_l, R_l = scipy.linalg.qr(A_l)
        Q_r, R_r = scipy.linalg.qr(A_r)

        Y = np.linalg.solve(R_l, Q_l.T @ B)
        
        Rr_inv = np.linalg.inv(R_r)
        
        XQr = Y @ Rr_inv
        
        X = XQr @ Q_r.T
        
        X = np.clip(X, 0, 1)  # Ensure values are in valid range

        if pad_width is not None:
            X = X[pad_width:-pad_width, pad_width:-pad_width]
            
    except np.linalg.LinAlgError as e:
        print(f"Linear algebra error in QR method: {str(e)}")
        X = np.zeros_like(B if pad_width is None else B[pad_width:-pad_width, pad_width:-pad_width])
        return X, time.time() - start_time
    except Exception as e:
        print(f"Unexpected error in QR method: {str(e)}")
        X = np.zeros_like(B if pad_width is None else B[pad_width:-pad_width, pad_width:-pad_width])
        return X, time.time() - start_time

    return X, time.time() - start_time

In [ ]:
"""
2.2 Test both deblurred methods and compare their results
"""
def test_deblur_methods(blurred_path, original_path, A_l_path, A_r_path):

    # Load images and kernels
    B = load_image(blurred_path)
    X_true = load_image(original_path)
    A_l = np.load(A_l_path)
    A_r = np.load(A_r_path)

    print(f"\nTesting deblurring methods on {blurred_path} with padding")
    print(f"Image shape: {X_true.shape}")

    # Check condition numbers
    cond_l = np.linalg.cond(A_l)
    cond_r = np.linalg.cond(A_r)
    print(f"Condition numbers - A_l: {cond_l:.2e}, A_r: {cond_r:.2e}")

    base_name = original_path.split('/')[1].split('_')[0]
    k_r = blurred_path.split('kr')[-1].split('.')[0]

    try:
        pad_info = np.load(f'output2/{base_name}_pad_info_kr{k_r}.npy', allow_pickle=True).item()
        pad_width = pad_info['pad_width']
        print(f"info pad_width: {pad_width}")
   
    except:
        n0 = X_true.shape[0]
        n1 = B.shape[0]
        pad_width = (n1 - n0)//2 if n1 != n0 else None
        print(f"manual pad_width: {pad_width}")
    
    # Apply LU method
    print("\nLU Deblurring:")
    X_lu, time_lu = deblur_lu(B, A_l, A_r,pad_width)
    error_lu = np.linalg.norm(X_lu - X_true) / np.linalg.norm(X_true)
    psnr_lu = compute_psnr(X_true, X_lu)
    print(f"Time: {time_lu:.3f}s")
    print(f"Relative Error: {error_lu:.3e}")
    print(f"PSNR: {psnr_lu:.2f} dB")

    # Apply QR method
    print("\nQR Deblurring:")
    X_qr, time_qr = deblur_qr(B, A_l, A_r,pad_width)
    error_qr = np.linalg.norm(X_qr - X_true) / np.linalg.norm(X_true)
    psnr_qr = compute_psnr(X_true, X_qr)
    print(f"Time: {time_qr:.3f}s")
    print(f"Relative Error: {error_qr:.3e}")
    print(f"PSNR: {psnr_qr:.2f} dB")

    # Visualization
    plt.figure(figsize=(15, 5))

    plt.subplot(141)
    plt.imshow(X_true, cmap='gray')
    plt.title('Original')
    plt.axis('off')

    plt.subplot(142)
    plt.imshow(B, cmap='gray')
    plt.title(f'Blurred\n k_r = {k_r} ')
    plt.axis('off')

    plt.subplot(143)
    plt.imshow(X_lu, cmap='gray')
    plt.title(f'LU Deblurred\nPSNR: {psnr_lu:.2f} dB')
    plt.axis('off')

    plt.subplot(144)
    plt.imshow(X_qr, cmap='gray')
    plt.title(f'QR Deblurred\nPSNR: {psnr_qr:.2f} dB')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    return {
        'LU': {'reconstruction': X_lu, 'time': time_lu, 'error': error_lu, 'psnr': psnr_lu},
        'QR': {'reconstruction': X_qr, 'time': time_qr, 'error': error_qr, 'psnr': psnr_qr}
    }

In [ ]:
# Test parameters
test_cases = [
    {
        'blurred': 'output2/256_blurred_with_padding_kr48.png',
        'original': 'test_images/256_hand_original.png',
        'A_l': 'output2/kernel_left_256_kr48.npy',
        'A_r': 'output2/kernel_right_256_kr48.npy'
    },
    {
        'blurred': 'output2/256_blurred_with_padding_kr96.png',
        'original': 'test_images/256_hand_original.png',
        'A_l': 'output2/kernel_left_256_kr96.npy',
        'A_r': 'output2/kernel_right_256_kr96.npy'
    },
    {
        'blurred': 'output2/1024_blurred_with_padding_kr48.png',
        'original': 'test_images/1024_books_original.png',
        'A_l': 'output2/kernel_left_1024_kr48.npy',
        'A_r': 'output2/kernel_right_1024_kr48.npy'
    },
    {
        'blurred': 'output2/1024_blurred_with_padding_kr96.png',
        'original': 'test_images/1024_books_original.png',
        'A_l': 'output2/kernel_left_1024_kr96.npy',
        'A_r': 'output2/kernel_right_1024_kr96.npy'
    },
    {
        'blurred': 'output2/1250_blurred_with_padding_kr48.png',
        'original': 'test_images/1250_m1_original.png',
        'A_l': 'output2/kernel_left_1250_kr48.npy',
        'A_r': 'output2/kernel_right_1250_kr48.npy'
    },
    {
        'blurred': 'output2/1250_blurred_with_padding_kr96.png',
        'original': 'test_images/1250_m1_original.png',
        'A_l': 'output2/kernel_left_1250_kr96.npy',
        'A_r': 'output2/kernel_right_1250_kr96.npy'
    },
    {
        'blurred': 'output2/3500_blurred_with_padding_kr48.png',
        'original': 'test_images/3500_garlic_original.png',
        'A_l': 'output2/kernel_left_3500_kr48.npy',
        'A_r': 'output2/kernel_right_3500_kr48.npy'
    },
    {
        'blurred': 'output2/3500_blurred_with_padding_kr96.png',
        'original': 'test_images/3500_garlic_original.png',
        'A_l': 'output2/kernel_left_3500_kr96.npy',
        'A_r': 'output2/kernel_right_3500_kr96.npy'
    }
]

# Run tests
results = []
for case in test_cases:
    result = test_deblur_methods(
        case['blurred'],
        case['original'],
        case['A_l'],
        case['A_r']
    )
    results.append(result)

In [ ]:
"""
We also test cases without padding, the results are similar to cases above
"""
test_cases_without_pad = [
    {
        'blurred': 'output2/256_blurred_kr48.png',
        'original': 'test_images/256_hand_original.png',
        'A_l': 'output2/kernel_left_256_kr48.npy',
        'A_r': 'output2/kernel_right_256_kr48.npy'
    },
    {
        'blurred': 'output2/256_blurred_kr96.png',
        'original': 'test_images/256_hand_original.png',
        'A_l': 'output2/kernel_left_256_kr96.npy',
        'A_r': 'output2/kernel_right_256_kr96.npy'
    },
    {
        'blurred': 'output2/1024_blurred_kr48.png',
        'original': 'test_images/1024_books_original.png',
        'A_l': 'output2/kernel_left_1024_kr48.npy',
        'A_r': 'output2/kernel_right_1024_kr48.npy'
    },
    {
        'blurred': 'output2/1024_blurred_kr96.png',
        'original': 'test_images/1024_books_original.png',
        'A_l': 'output2/kernel_left_1024_kr96.npy',
        'A_r': 'output2/kernel_right_1024_kr96.npy'
    },
    {
        'blurred': 'output2/1250_blurred_kr48.png',
        'original': 'test_images/1250_m1_original.png',
        'A_l': 'output2/kernel_left_1250_kr48.npy',
        'A_r': 'output2/kernel_right_1250_kr48.npy'
    },
    {
        'blurred': 'output2/1250_blurred_kr96.png',
        'original': 'test_images/1250_m1_original.png',
        'A_l': 'output2/kernel_left_1250_kr96.npy',
        'A_r': 'output2/kernel_right_1250_kr96.npy'
    },
    {
        'blurred': 'output2/3500_blurred_kr48.png',
        'original': 'test_images/3500_garlic_original.png',
        'A_l': 'output2/kernel_left_3500_kr48.npy',
        'A_r': 'output2/kernel_right_3500_kr48.npy'
    },
    {
        'blurred': 'output2/3500_blurred_kr96.png',
        'original': 'test_images/3500_garlic_original.png',
        'A_l': 'output2/kernel_left_3500_kr96.npy',
        'A_r': 'output2/kernel_right_3500_kr96.npy'
    }
]

# Run tests
results_without_pad = []
for case in test_cases_without_pad:
    result = test_deblur_methods(
        case['blurred'],
        case['original'],
        case['A_l'],
        case['A_r']
    )
    results_without_pad.append(result)

In [ ]:
"""
2.3 Compare and discussed the performance of LU and QR decomposition.
"""

def systematic_comparison(image_path, k_values):
    """Systematically compare performances of LU and QR under different k values"""
    results = []
    n = int(image_path.split('/')[1].split('_')[0])
    X_true = load_image(image_path)

    for k_r in k_values:
 
        X, B, B_padded, A_l_pad, A_r_pad, padded_width = create_blurred_image(
            image_path, n=n, k_l=12, k_r=k_r, pad=True
        )
        
        X, B, _, A_l, A_r, padded_width = create_blurred_image(
            image_path, n=n, k_l=12, k_r=k_r, pad=False
        )

        X_lu, time_lu = deblur_lu(B, A_l, A_r,padded_width)

        lu_analysis = {
            'condition_numbers': {
                'A_l': np.linalg.cond(A_l),
                'A_r': np.linalg.cond(A_r),
                'A_l_padded': np.linalg.cond(A_l_pad) if A_l_pad is not None else None,
                'A_r_padded': np.linalg.cond(A_r_pad) if A_r_pad is not None else None
            },
            'residual': {
                'relative': np.linalg.norm(A_l @ X_lu @ A_r - B, 'fro') / np.linalg.norm(B, 'fro')
            }
        }

        X_qr, time_qr = deblur_qr(B, A_l, A_r,padded_width)
        qr_analysis = {
            'condition_numbers': {
                'A_l': np.linalg.cond(A_l),
                'A_r': np.linalg.cond(A_r),
                'A_l_padded': np.linalg.cond(A_l_pad) if A_l_pad is not None else None,
                'A_r_padded': np.linalg.cond(A_r_pad) if A_r_pad is not None else None
            },
            'residual': {
                'relative': np.linalg.norm(A_l @ X_qr @ A_r - B, 'fro') / np.linalg.norm(B, 'fro')
            }
        }

        results.append({
            'k_r': k_r,
            'kernels': {
                'original': {'A_l': A_l, 'A_r': A_r},
                'padded': {'A_l': A_l_pad, 'A_r': A_r_pad}
            },
            'LU': {
                'time': time_lu,
                'psnr': compute_psnr(X_true, X_lu),
                'analysis': lu_analysis
            },
            'QR': {
                'time': time_qr,
                'psnr': compute_psnr(X_true, X_qr),
                'analysis': qr_analysis
            }
        })

    return results


def plot_comparison_results(results):

    k_values = [r['k_r'] for r in results]

    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # PSNR
    axes[0,0].plot(k_values, [r['LU']['psnr'] for r in results], 'b-', label='LU')
    axes[0,0].plot(k_values, [r['QR']['psnr'] for r in results], 'r-', label='QR')
    axes[0,0].set_title('PSNR vs k_r')
    axes[0,0].set_xlabel('k_r')
    axes[0,0].set_ylabel('PSNR (dB)')
    axes[0,0].legend()

    # Conditional Numbers
    axes[0,1].plot(k_values, [r['LU']['analysis']['condition_numbers']['A_r'] for r in results])
    axes[0,1].set_title('Condition Number of A_r vs k_r')
    axes[0,1].set_xlabel('k_r')
    axes[0,1].set_ylabel('Condition Number')
    axes[0,1].set_yscale('log')

    # Comparison Errors
    axes[1,0].plot(k_values, [r['LU']['analysis']['residual']['relative'] for r in results], 'b-', label='LU')
    axes[1,0].plot(k_values, [r['QR']['analysis']['residual']['relative'] for r in results], 'r-', label='QR')
    axes[1,0].set_title('Relative Residual vs k_r')
    axes[1,0].set_xlabel('k_r')
    axes[1,0].set_ylabel('Relative Residual')
    axes[1,0].legend()

    # CPU Times
    axes[1,1].plot(k_values, [r['LU']['time'] for r in results], 'b-', label='LU')
    axes[1,1].plot(k_values, [r['QR']['time'] for r in results], 'r-', label='QR')
    axes[1,1].set_title('Computation Time vs k_r')
    axes[1,1].set_xlabel('k_r')
    axes[1,1].set_ylabel('Time (s)')
    axes[1,1].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

test_images = [
    'test_images/1250_m1_original.png'    
]

k_values = [48, 72, 96, 120, 150, 200]  


all_results = {}

for img_path in test_images:
    print(f"\nTesting image: {img_path}")
    results = systematic_comparison(img_path, k_values)
    img_name = img_path.split('/')[-1].split('_')[0]
    all_results[img_name] = results

    plot_comparison_results(results)

    # 打印详细结果
    print(f"\nDetailed results for {img_name}:")
    for r in results:
        print(f"\nk_r = {r['k_r']}:")
        print(f"LU - PSNR: {r['LU']['psnr']:.2f} dB, Time: {r['LU']['time']:.3f}s")
        print(f"QR - PSNR: {r['QR']['psnr']:.2f} dB, Time: {r['QR']['time']:.3f}s")
        print(f"Condition number of A_r: {r['LU']['analysis']['condition_numbers']['A_r']:.2e}")

In [ ]:
"""
part(c) 

Compute the QR factorization of matrix A using Householder reflections.
Strictly following the lecture notes.
"""


In [ ]:
def pivoting(A):
    m, n = A.shape
    P = np.eye(m)
    A_pivoted = A.copy()

    for k in range(min(m, n)):
        # Find the index of the largest absolute value in column k
        max_index = np.argmax(np.abs(A_pivoted[k:, k])) + k

        if max_index != k:
            # Swap rows in A
            A_pivoted[[k, max_index]] = A_pivoted[[max_index, k]]
            # Swap corresponding rows in P (permutation matrix)
            P[[k, max_index]] = P[[max_index, k]]

    return P, A_pivoted

def householder_reflection(x):
    x = np.asarray(x, dtype=float)
    v = np.zeros_like(x)
    alpha = np.linalg.norm(x)
    if x[0] <= 0:
        v[0] = x[0] - alpha
    else:
        v[0] = x[0] + alpha
    v[1:] = x[1:]
    beta = 2 / (np.linalg.norm(v) ** 2)
    return v, beta

def qr_with_permutation(A):
    m, n = A.shape
    Q = np.eye(m)

    # Apply column pivoting
    P, R = pivoting(np.copy(A))

    # Perform Householder reflections to compute Q and R
    for k in range(min(m, n)):  # Use min(m, n) to avoid out-of-bounds error
        x = R[k:, k]
        v, beta = householder_reflection(x)
        H = np.eye(m)
        H[k:, k:] -= beta * np.outer(v, v)
        R = np.dot(H, R)
        Q = np.dot(Q, H)

    # Reorder Q based on P
    Q = np.dot(Q, P.T)

    return Q, R, P

def solve_with_qr(A, B, use_permutation=False):
    if not use_permutation:
        Q, R = np.linalg.qr(A)
        y = Q.T @ B
        x = np.linalg.inv(R) @ y
    else:
        Q, R, P = qr_with_permutation(A)
        y = Q.T @ B
        x = P @ np.linalg.inv(R) @ y
    return x

def deblur_householder_qr(B, A_l, A_r, use_permutation=False,pad_width=None):
    start_time = time.time()
    
    C = solve_with_qr(A_l, B, use_permutation)
    X = solve_with_qr(A_r.T, C.T, use_permutation).T
    X = np.clip(X, 0, 1)

    if pad_width is not None:
        X = X[pad_width:-pad_width, pad_width:-pad_width]

    cpu_time = time.time() - start_time
    
    return X, cpu_time


In [ ]:
"""
part(d) Implement and test Householder QR decomposition
"""

In [ ]:
def test_custom_qr(image_paths, k_r_values):
    """Test custom QR implementation on test images"""
    for path in image_paths:
        print(f"\nProcessing {path}")
        n = int(path.split('/')[1].split('_')[0])

        for k_r in k_r_values:
            # Create blurred image
            X, B, B_padded, A_l, A_r, pad_width = create_blurred_image(
                path, n=n, k_l=12, k_r=k_r, pad=True
            )

            # Apply custom QR deblurring
            X_recon, cpu_time = deblur_householder_qr(B_padded, A_l, A_r,use_permutation=True, pad_width=pad_width)
            psnr = compute_psnr(X, X_recon)

            # Plot results
            plt.figure(figsize=(12, 4))

            plt.subplot(131)
            plt.imshow(X, cmap='gray')
            plt.title('Original')
            plt.axis('off')

            plt.subplot(132)
            plt.imshow(B, cmap='gray')
            plt.title('Blurred')
            plt.axis('off')

            plt.subplot(133)
            plt.imshow(X_recon, cmap='gray')
            plt.title(f'Reconstructed\nPSNR: {psnr:.2f}dB\nTime: {cpu_time:.3f}s')
            plt.axis('off')

            plt.suptitle(f'Custom QR Deblurring - Image size: {n}x{n}, k_r={k_r}')
            plt.tight_layout()
            plt.show()

            print(f"\nResults for {path} with k_r={k_r}:")
            print(f"PSNR: {psnr:.2f}dB")
            print(f"CPU Time: {cpu_time:.3f}s")

            # Compute relative error
            rel_error = np.linalg.norm(X_recon - X, 'fro') / np.linalg.norm(X, 'fro')
            print(f"Relative Error: {rel_error:.3e}")

# Test parameters
test_image_640 = [
    'test_images/640_lion_original.png'
]
k_r_values_640 = [48, 72, 96]

test_image_1250 = [
    'test_images/1250_m1_original.png'
]
k_r_values_1250 = [24, 48]

# Run the test
test_custom_qr(test_image_640, k_r_values_640)
test_custom_qr(test_image_1250, k_r_values_1250)

In [ ]:
"""
part(e)
    Implement and analyze improvements for the deblurring problem:
    1. Matrix condition analysis
    2. Different regularization approaches
    3. Least squares formulation
"""

In [ ]:
"""
 1. Optimize QR Deblurring Method using Tikhonov Regularization
"""

def solve_least_squares(B, A_l, A_r, lambda_reg=1e-6):
    """
    Solve min ||A_l X A_r - B||_F^2 + lambda||X||_F^2
    using normal equations
    """
    m, n = B.shape

    # Form AᵀA and Aᵀb without explicitly forming Kronecker product
    def matvec(x):
        """Compute (Aᵣᵀ ⊗ Aₗ)ᵀ(Aᵣᵀ ⊗ Aₗ)x efficiently"""
        X = x.reshape(m, n)
        Y = A_l.T @ (A_l @ X @ A_r) @ A_r.T + lambda_reg * X
        return Y.reshape(-1)

    # Form right-hand side without explicitly forming Kronecker product
    rhs = A_l.T @ B @ A_r.T
    rhs = rhs.reshape(-1)

    # Solve normal equations using conjugate gradient
    x, info = splinalg.cg(
        splinalg.LinearOperator((m*n, m*n), matvec=matvec),
        rhs,
        tol=1e-10,
        maxiter=10000
    )

    if info != 0:
        print(f"Warning: CG did not converge, info={info}")

    # Reshape solution back to matrix form
    X = x.reshape(m, n)
    return np.clip(X, 0, 1)


def test_least_squares(test_cases):
    """Test least squares solution"""
    for case in test_cases:
        print(f"\nTesting {case['blurred']}")

        # Load data
        B = load_image(case['blurred'])
        X_true = load_image(case['original'])
        A_l = np.load(case['A_l'])
        A_r = np.load(case['A_r'])

        # Get padding info first
        base_name = case['original'].split('/')[1].split('_')[0]
        k_r = case['blurred'].split('kr')[-1].split('.')[0]
        try:
            pad_info = np.load(f'output2/{base_name}_pad_info_kr{k_r}.npy', allow_pickle=True).item()
            pad_width = pad_info['pad_width']
        except:
            n0 = X_true.shape[0]
            n1 = B.shape[0]
            pad_width = (n1 - n0)//2 if n1 != n0 else None

        print(f"Padding width: {pad_width}")

        # Analyze condition numbers
        cond_l = np.linalg.cond(A_l)
        cond_r = np.linalg.cond(A_r)
        print(f"Condition numbers - A_l: {cond_l:.2e}, A_r: {cond_r:.2e}")

        # Try different regularization parameters
        lambda_values = [1e-4, 1e-6, 1e-8]
        results = []

        for lambda_reg in lambda_values:
            print(f"\nSolving with λ = {lambda_reg}")
            start_time = time.time()
            X_full = solve_least_squares(B, A_l, A_r, lambda_reg)
            solve_time = time.time() - start_time

            # Compute residual on full image
            residual = np.linalg.norm(A_l @ X_full @ A_r - B, 'fro')

            # Remove padding for visualization and PSNR computation
            if pad_width is not None:
                X = X_full[pad_width:-pad_width, pad_width:-pad_width]
            else:
                X = X_full

            # Compute PSNR
            psnr = compute_psnr(X_true, X)
            results.append((lambda_reg, X, solve_time, psnr, residual))

        # Plot results
        plt.figure(figsize=(5*(len(results) + 2), 5))

        # Plot original
        plt.subplot(1, len(results)+2, 1)
        plt.imshow(X_true, cmap='gray')
        plt.title('Original')
        plt.axis('off')

        # Plot blurred
        plt.subplot(1, len(results)+2, 2)
        if pad_width is not None:
            plt.imshow(B[pad_width:-pad_width, pad_width:-pad_width], cmap='gray')
        else:
            plt.imshow(B, cmap='gray')
        plt.title('Blurred')
        plt.axis('off')

        # Plot results
        for i, (lambda_reg, X, solve_time, psnr, residual) in enumerate(results, start=3):
            plt.subplot(1, len(results)+2, i)
            plt.imshow(X, cmap='gray')
            plt.title(f'λ={lambda_reg:.0e}\nPSNR: {psnr:.2f}dB\n'
                      f'Time: {solve_time:.2f}s\n'
                      f'Residual: {residual:.2e}')
            plt.axis('off')

        plt.tight_layout()
        plt.show()

# Test parameters
test_cases = [
    {
        'blurred': 'output2/256_blurred_with_padding_kr48.png',
        'original': 'test_images/256_hand_original.png',
        'A_l': 'output2/kernel_left_256_kr48.npy',
        'A_r': 'output2/kernel_right_256_kr48.npy'
    }
]

# Run proper least squares solution
test_least_squares(test_cases)

In [ ]:
"""
2. 
 Auto adaptive regularization parameters
    
    The regularization parameters are selected adaptively according to the problem characteristics:
    i)  The base lambda value is based on the condition number
    ii) The ill-posed problem requires stronger regularization, adjust the base lambda by multiplying the adjustment coefficient
    iii) Make sure that lambda is within a reasonable range
    
"""
def deblur_qr_regularized(B, A_l, A_r, lambda_reg=1e-1,pad_width=None):
    try:
        start_time = time.time()

        Q_l, R_l = linalg.qr(A_l)
        Q_r, R_r = linalg.qr(A_r)

        Y1 = Q_l.T @ B
        Z = solve(R_l.T @ R_l + lambda_reg * np.eye(R_l.shape[1]),
                  R_l.T @ Y1,
                  check_finite=True)

        Y2 = Q_r.T @ Z.T
        X = solve(R_r.T @ R_r + lambda_reg * np.eye(R_r.shape[1]),
                  R_r.T @ Y2,
                  check_finite=True).T

        X = np.clip(X, 0, 1)  
        
        if pad_width is not None:
            X = X[pad_width:-pad_width, pad_width:-pad_width]
        
        return X, time.time() - start_time

    except Exception as e:
        print(f"Error in regularized QR method: {str(e)}")
        return np.zeros_like(B), 0

def get_reg_parameter(A_l, A_r):

    cond_l = np.linalg.cond(A_l)
    cond_r = np.linalg.cond(A_r)
    n = A_l.shape[0]

    base_lambda = 1.0 / np.sqrt(max(cond_l, cond_r))

    if max(cond_l, cond_r) > 1e10:
        base_lambda *= 1e5
    elif max(cond_l, cond_r) > 1e5:
        base_lambda *= 1e3

    size_factor = np.log10(n) / 3
    base_lambda *= size_factor

    lambda_reg = np.clip(base_lambda, 1e-6, 1e-2)

    print(f"Condition numbers - A_l: {cond_l:.2e}, A_r: {cond_r:.2e}")
    print(f"Selected lambda: {lambda_reg:.2e}")

    return lambda_reg

def deblur_qr_auto(B, A_l, A_r,pad_width):

    lambda_reg = get_reg_parameter(A_l, A_r)

    return deblur_qr_regularized(B, A_l, A_r, lambda_reg, pad_width)

# Test for the auto adaptive Tikhonov Regularization
def test_adaptive_qr(test_cases):

    for case in test_cases:

        print(f"\nTesting {case['blurred']}")
        B = load_image(case['blurred'])
        X_true = load_image(case['original'])
        A_l = np.load(case['A_l'])
        A_r = np.load(case['A_r'])

        base_name = case['original'].split('/')[1].split('_')[0]
        k_r = case['blurred'].split('kr')[-1].split('.')[0]
        
        try:
            pad_info = np.load(f'output3/{base_name}_pad_info_kr{k_r}.npy', allow_pickle=True).item()
            pad_width = pad_info['pad_width']
            print(f"info pad_width: {pad_width}")
        except:
           
            n0 = X_true.shape[0]
            n1 = B.shape[0]
            pad_width = (n1 - n0)//2 if n1 != n0 else None
            print(f"manual pad_width: {pad_width}")
       
        X_lu, time_lu = deblur_lu(B, A_l, A_r,pad_width)
        psnr_lu = compute_psnr(X_true, X_lu)

        X_qr, time_qr = deblur_qr_auto(B, A_l, A_r,pad_width)
        psnr_qr = compute_psnr(X_true, X_qr)

        plt.figure(figsize=(15, 5))

        plt.subplot(141)
        plt.imshow(X_true, cmap='gray')
        plt.title('Original')
        plt.axis('off')

        plt.subplot(142)
        plt.imshow(B, cmap='gray')
        plt.title('Blurred')
        plt.axis('off')

        plt.subplot(143)
        plt.imshow(X_lu, cmap='gray')
        plt.title(f'LU\nPSNR: {psnr_lu:.2f} dB')
        plt.axis('off')

        plt.subplot(144)
        plt.imshow(X_qr, cmap='gray')
        plt.title(f'Adaptive QR\nPSNR: {psnr_qr:.2f} dB')
        plt.axis('off')

        plt.tight_layout()
        plt.show()


test_cases = [
    {
        'blurred': 'output3/640_blurred_with_padding_kr48.png',
        'original': 'output3/640_original.png',
        'A_l': 'output3/kernel_left_640_kr48.npy',
        'A_r': 'output3/kernel_right_640_kr48.npy'
    },
    {
        'blurred': 'output3/640_blurred_with_padding_kr96.png',
        'original': 'output3/640_original.png',
        'A_l': 'output3/kernel_left_640_kr96.npy',
        'A_r': 'output3/kernel_right_640_kr96.npy'
    },
    {
        'blurred': 'output3/1024_blurred_with_padding_kr48.png',
        'original': 'output3/1024_original.png',
        'A_l': 'output3/kernel_left_1024_kr48.npy',
        'A_r': 'output3/kernel_right_1024_kr48.npy'
    },
    {
        'blurred': 'output3/1024_blurred_with_padding_kr96.png',
        'original': 'output3/1024_original.png',
        'A_l': 'output3/kernel_left_1024_kr96.npy',
        'A_r': 'output3/kernel_right_1024_kr96.npy'
    },
    
    {
        'blurred': 'output3/1250_blurred_with_padding_kr48.png',
        'original': 'output3/1250_original.png',
        'A_l': 'output3/kernel_left_1250_kr48.npy',
        'A_r': 'output3/kernel_right_1250_kr48.npy'
    },
    {
        'blurred': 'output3/1250_blurred_with_padding_kr96.png',
        'original': 'output3/1250_original.png',
        'A_l': 'output3/kernel_left_1250_kr96.npy',
        'A_r': 'output3/kernel_right_1250_kr96.npy'
    },
    {
        'blurred': 'output3/3500_blurred_with_padding_kr48.png',
        'original': 'output3/3500_original.png',
        'A_l': 'output3/kernel_left_3500_kr48.npy',
        'A_r': 'output3/kernel_right_3500_kr48.npy'
    },
    {
        'blurred': 'output3/3500_blurred_with_padding_kr96.png',
        'original': 'output3/3500_original.png',
        'A_l': 'output3/kernel_left_3500_kr96.npy',
        'A_r': 'output3/kernel_right_3500_kr96.npy'
    }
]

test_adaptive_qr(test_cases)

In [ ]:
"""
3. Iterative deblurring with adaptive regularization and Truncated SVD
"""

def iterative_deblur(B, A_l, A_r, lambda_reg=1e-3, max_iter=50, tol=1e-4):
    
    X = np.zeros_like(B)
    prev_X = np.zeros_like(B)

    # Precompute matrices for efficiency
    A_l_T = A_l.T
    A_r_T = A_r.T

    # Gradient descent with momentum
    momentum = 0.9
    v = np.zeros_like(B)
    step_size = 0.1  # Initial step size

    for i in range(max_iter):
        # Compute residual
        residual = A_l @ X @ A_r - B

        # Compute gradient
        gradient = A_l_T @ residual @ A_r_T + lambda_reg * X

        # Update with momentum
        v = momentum * v - step_size * gradient
        X = X + v

        # Compute relative change
        rel_change = np.linalg.norm(X - prev_X) / np.linalg.norm(X)
        if rel_change < tol:
            print(f"Converged after {i+1} iterations")
            break

        prev_X = X.copy()

        # Project to ensure valid image values
        X = np.clip(X, 0, 1)

    return X

def trunc_level_adaptive(s):
    """Determine truncation level adaptively based on singular values"""
    # Find where singular values drop significantly
    ratios = s[1:] / s[:-1]
    drop_idx = np.where(ratios < 0.1)[0]
    if len(drop_idx) > 0:
        return drop_idx[0] + 1
    return len(s) // 2  # Default to half if no clear drop

def truncated_svd_deblur(B, A_l, A_r, trunc_ratio=0.5):
    """Deblurring using truncated SVD with consistent dimensions"""
    U_l, s_l, Vh_l = np.linalg.svd(A_l, full_matrices=False)
    U_r, s_r, Vh_r = np.linalg.svd(A_r, full_matrices=False)

    # Determine truncation rank adaptively
    n = A_l.shape[0]
    k_max = min(n, int(n * trunc_ratio))
    k_l = min(k_max, len(s_l))
    k_r = min(k_max, len(s_r))
    k = min(k_l, k_r)

    print(f"Using rank {k} out of {n} possible components")

    # Truncate components
    U_l_t = U_l[:, :k]
    s_l_t = s_l[:k]
    Vh_l_t = Vh_l[:k, :]

    U_r_t = U_r[:, :k]
    s_r_t = s_r[:k]
    Vh_r_t = Vh_r[:k, :]

    # Print shapes for debugging
    print(f"U_l_t shape: {U_l_t.shape}")
    print(f"Vh_l_t shape: {Vh_l_t.shape}")
    print(f"U_r_t shape: {U_r_t.shape}")
    print(f"Vh_r_t shape: {Vh_r_t.shape}")
    print(f"B shape: {B.shape}")

    # Transform B with regularization
    B_transformed = U_l_t.T @ B @ U_r_t

    # Create regularization matrix
    outer_prod = np.outer(s_l_t, s_r_t)
    reg_matrix = outer_prod / (outer_prod**2 + 1e-10)

    # Solve the transformed system
    X_transformed = B_transformed * reg_matrix

    # Transform back
    X = U_l_t @ X_transformed @ U_r_t.T

    return np.clip(X, 0, 1)

def test_improved_methods(test_cases):
    """Test both deblurring methods"""
    for case in test_cases:
        print(f"\nTesting {case['blurred']}")

        # Load data
        B = load_image(case['blurred'])
        X_true = load_image(case['original'])
        A_l = np.load(case['A_l'])
        A_r = np.load(case['A_r'])

        # Get padding info and handle padding
        base_name = case['original'].split('/')[1].split('_')[0]
        k_r = case['blurred'].split('kr')[-1].split('.')[0]

        try:
            pad_info = np.load(f'output2/{base_name}_pad_info_kr{k_r}.npy', allow_pickle=True).item()
            pad_width = pad_info['pad_width']
        except:
            n0 = X_true.shape[0]
            n1 = B.shape[0]
            pad_width = (n1 - n0)//2 if n1 != n0 else None

        # Test iterative method
        print("\nTesting iterative method...")
        start_time = time.time()
        X_iter = iterative_deblur(B, A_l, A_r)
        time_iter = time.time() - start_time

        print("\nTesting SVD method...")
        start_time = time.time()
        X_svd = truncated_svd_deblur(B, A_l, A_r)
        time_svd = time.time() - start_time

        # Handle padding
        if pad_width is not None:
            X_iter = X_iter[pad_width:-pad_width, pad_width:-pad_width]
            X_svd = X_svd[pad_width:-pad_width, pad_width:-pad_width]

        # Compute metrics
        psnr_iter = compute_psnr(X_true, X_iter)
        psnr_svd = compute_psnr(X_true, X_svd)

        # Plot results
        plt.figure(figsize=(15, 5))

        plt.subplot(141)
        plt.imshow(X_true, cmap='gray')
        plt.title('Original')
        plt.axis('off')

        plt.subplot(142)
        plt.imshow(B, cmap='gray')
        plt.title('Blurred')
        plt.axis('off')

        plt.subplot(143)
        plt.imshow(X_iter, cmap='gray')
        plt.title(f'Iterative\nPSNR: {psnr_iter:.2f}dB\nTime: {time_iter:.2f}s')
        plt.axis('off')

        plt.subplot(144)
        plt.imshow(X_svd, cmap='gray')
        plt.title(f'Truncated SVD\nPSNR: {psnr_svd:.2f}dB\nTime: {time_svd:.2f}s')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

# Test parameters
test_cases = [
    {
        'blurred': 'output3/640_blurred_with_padding_kr48.png',
        'original': 'output3/640_original.png',
        'A_l': 'output3/kernel_left_640_kr48.npy',
        'A_r': 'output3/kernel_right_640_kr48.npy'
    },
    {
        'blurred': 'output3/640_blurred_with_padding_kr96.png',
        'original': 'output3/640_original.png',
        'A_l': 'output3/kernel_left_640_kr96.npy',
        'A_r': 'output3/kernel_right_640_kr96.npy'
    },
    {
        'blurred': 'output3/1024_blurred_with_padding_kr48.png',
        'original': 'output3/1024_original.png',
        'A_l': 'output3/kernel_left_1024_kr48.npy',
        'A_r': 'output3/kernel_right_1024_kr48.npy'
    },
    {
        'blurred': 'output3/1024_blurred_with_padding_kr96.png',
        'original': 'output3/1024_original.png',
        'A_l': 'output3/kernel_left_1024_kr96.npy',
        'A_r': 'output3/kernel_right_1024_kr96.npy'
    },

    {
        'blurred': 'output3/1250_blurred_with_padding_kr48.png',
        'original': 'output3/1250_original.png',
        'A_l': 'output3/kernel_left_1250_kr48.npy',
        'A_r': 'output3/kernel_right_1250_kr48.npy'
    },
    {
        'blurred': 'output3/1250_blurred_with_padding_kr96.png',
        'original': 'output3/1250_original.png',
        'A_l': 'output3/kernel_left_1250_kr96.npy',
        'A_r': 'output3/kernel_right_1250_kr96.npy'
    },
    {
        'blurred': 'output3/3500_blurred_with_padding_kr48.png',
        'original': 'output3/3500_original.png',
        'A_l': 'output3/kernel_left_3500_kr48.npy',
        'A_r': 'output3/kernel_right_3500_kr48.npy'
    },
    {
        'blurred': 'output3/3500_blurred_with_padding_kr96.png',
        'original': 'output3/3500_original.png',
        'A_l': 'output3/kernel_left_3500_kr96.npy',
        'A_r': 'output3/kernel_right_3500_kr96.npy'
    }
]

# Run the improved methods
test_improved_methods(test_cases)